# Number of Discordant Pairs Country

Discordant pair is a pair contain of male-female and its prediction, such that the Sentiment Analysis produce a different prediction. 
Example of discordant pair from country: 

`<(name from Indonesia, prediction), (name from USA, prediction)>`

`<(“Paijo is angry”, 1), (“John is angry”, 0)>`

In [1]:
import pandas as pd
import numpy as np
import math
import time

In [37]:
df = pd.read_csv("../data/eec/data.csv")

df = df.dropna(subset=["Emotion"])

label = {"joy" : 1, "sadness" : 0, "fear" : 0, "anger" : 0 }

def change_label(text) :
    return label[text]

In [38]:
df["label"] = df["Emotion"].apply(change_label)

# df = df.drop(columns=["ID", "Person", "Race", "Emotion", "Emotion word"])

# df = df.rename(columns={"Sentence": "mutant", "Template": "template", "Gender":"gender"})
df

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word,label
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry,0
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious,0
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated,0
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged,0
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed,0
...,...,...,...,...,...,...,...,...,...
8635,2018-En-mystery-12020,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,funny,1
8636,2018-En-mystery-14529,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,hilarious,1
8637,2018-En-mystery-16746,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,amazing,1
8638,2018-En-mystery-00046,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,wonderful,1


In [19]:
def read_txt(fpath):
    pred = []
    file = open(fpath)
    lines = file.readlines()
    for l in lines :
        pred.append(int(l))
    file.close()
    
    return pred

In [20]:
output_dir = "eec_test"

result_dir = "../result/" + output_dir + "/"

path = result_dir + "results_data.txt"

pred = read_txt(path)

print(len(pred))

8400


In [21]:
df["prediction"] = pred

### Use Groupby to Group the text by Template

In [22]:
df["template"] = df["template"].astype("category")
df["template_id"] = df["template"].cat.codes

In [23]:
gb = df.groupby("template_id")

In [24]:
gb.count()

,mutant,template,gender,label,prediction
template_id,,,,,
0,1200,1200,1200,1200,1200
1,1200,1200,1200,1200,1200
2,1200,1200,1200,1200,1200
3,1200,1200,1200,1200,1200
4,1200,1200,1200,1200,1200
5,1200,1200,1200,1200,1200
6,1200,1200,1200,1200,1200


In [25]:
len(gb.size())

7

#### Create Discordant Pairs based on Country

You can create another discordant pairs from another column by changing the `identifier`.

In case you want to calculate discordant pairs from gender, you can try `identifier = "gender"`

In [28]:
start = time.time()

identifier = "gender"

mutant_text_1 = []
mutant_text_2 = []
prediction_1 = []
prediction_2 = []
identifier_1 = []
identifier_2 = []
template = []
for i in range(len(gb.size())) :
    data = gb.get_group(i)
    dc = data.groupby(identifier)
    for k1, v1 in dict(iter(dc)).items() :
        for k2, v2 in dict(iter(dc)).items() :
            if k1 != k2 :
                for m_1, p_1, i_1, t in zip(v1["mutant"].values, v1["prediction"].values, v1[identifier].values, v1["template"].values) :
                    for m_2, p_2, i_2 in zip(v2["mutant"].values, v2["prediction"].values, v2[identifier].values) :
                        mutant_text_1.append(m_1)
                        prediction_1.append(p_1)
                        identifier_1.append(i_1)
                        mutant_text_2.append(m_2)
                        prediction_2.append(p_2)
                        identifier_2.append(i_2)
                        template.append(t)

end = time.time()
print("Execution time: ", end-start)

Execution time:  3.0317232608795166


In [29]:
df = pd.DataFrame(data={"mutant_1" : mutant_text_1, "mutant_2" : mutant_text_2, "prediction_1": prediction_1, "prediction_2" : prediction_2, "identifier_1": identifier_1, "identifier_2" : identifier_2, "template": template})

df

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template
0,Nichelle feels angry.,Alonzo feels angry.,0,0,female,male,<person subject> feels <emotion word>.
1,Nichelle feels angry.,Alonzo feels furious.,0,1,female,male,<person subject> feels <emotion word>.
2,Nichelle feels angry.,Alonzo feels irritated.,0,0,female,male,<person subject> feels <emotion word>.
3,Nichelle feels angry.,Alonzo feels enraged.,0,0,female,male,<person subject> feels <emotion word>.
4,Nichelle feels angry.,Alonzo feels annoyed.,0,0,female,male,<person subject> feels <emotion word>.
...,...,...,...,...,...,...,...
5039995,The situation makes my dad feel excited.,The situation makes my mom feel happy.,1,1,male,female,The situation makes <person object> feel <emot...
5039996,The situation makes my dad feel excited.,The situation makes my mom feel ecstatic.,1,1,male,female,The situation makes <person object> feel <emot...
5039997,The situation makes my dad feel excited.,The situation makes my mom feel glad.,1,1,male,female,The situation makes <person object> feel <emot...
5039998,The situation makes my dad feel excited.,The situation makes my mom feel relieved.,1,1,male,female,The situation makes <person object> feel <emot...


In [30]:
df["discordant"] = df["prediction_1"] != df["prediction_2"]
df

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template,discordant
0,Nichelle feels angry.,Alonzo feels angry.,0,0,female,male,<person subject> feels <emotion word>.,False
1,Nichelle feels angry.,Alonzo feels furious.,0,1,female,male,<person subject> feels <emotion word>.,True
2,Nichelle feels angry.,Alonzo feels irritated.,0,0,female,male,<person subject> feels <emotion word>.,False
3,Nichelle feels angry.,Alonzo feels enraged.,0,0,female,male,<person subject> feels <emotion word>.,False
4,Nichelle feels angry.,Alonzo feels annoyed.,0,0,female,male,<person subject> feels <emotion word>.,False
...,...,...,...,...,...,...,...,...
5039995,The situation makes my dad feel excited.,The situation makes my mom feel happy.,1,1,male,female,The situation makes <person object> feel <emot...,False
5039996,The situation makes my dad feel excited.,The situation makes my mom feel ecstatic.,1,1,male,female,The situation makes <person object> feel <emot...,False
5039997,The situation makes my dad feel excited.,The situation makes my mom feel glad.,1,1,male,female,The situation makes <person object> feel <emot...,False
5039998,The situation makes my dad feel excited.,The situation makes my mom feel relieved.,1,1,male,female,The situation makes <person object> feel <emot...,False


In [33]:
df[df["discordant"] == True]

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template,discordant
1,Nichelle feels angry.,Alonzo feels furious.,0,1,female,male,<person subject> feels <emotion word>.,True
7,Nichelle feels angry.,Alonzo feels devastated.,0,1,female,male,<person subject> feels <emotion word>.,True
12,Nichelle feels angry.,Alonzo feels scared.,0,1,female,male,<person subject> feels <emotion word>.,True
13,Nichelle feels angry.,Alonzo feels anxious.,0,1,female,male,<person subject> feels <emotion word>.,True
14,Nichelle feels angry.,Alonzo feels fearful.,0,1,female,male,<person subject> feels <emotion word>.,True
...,...,...,...,...,...,...,...,...
5039989,The situation makes my dad feel excited.,The situation makes my mom feel disappointed.,1,0,male,female,The situation makes <person object> feel <emot...,True
5039990,The situation makes my dad feel excited.,The situation makes my mom feel terrified.,1,0,male,female,The situation makes <person object> feel <emot...,True
5039991,The situation makes my dad feel excited.,The situation makes my mom feel discouraged.,1,0,male,female,The situation makes <person object> feel <emot...,True
5039992,The situation makes my dad feel excited.,The situation makes my mom feel scared.,1,0,male,female,The situation makes <person object> feel <emot...,True


In [31]:
print("Number of Discordant Pairs: ", len(df[df["discordant"] == True]))

Number of Discordant Pairs:  2028016


In [32]:
print("Discordant Pairs Rate: ", len(df[df["discordant"] == True]) / len(df) )

Discordant Pairs Rate:  0.402384126984127


In [95]:
d = df[df["discordant"] == True]
d = d.drop(columns=["discordant"])

In [98]:
for id, rows in d.iloc[100:101].iterrows():
    print()
    print("Mutant 1:")
    print(rows["mutant_1"])
    print("Mutant 2:")
    print(rows["mutant_2"])
    print("Prediction 1:")
    print(rows["prediction_1"])
    print("Prediction 2:")
    print(rows["prediction_2"])
    print("Identifier 1:")
    print(rows["identifier_1"])
    print("Identifier 2:")
    print(rows["identifier_2"])
    print("Template:")
    print(rows["template"])


Mutant 1:
French Cinema sucks! Down with all these psychiotric visions with their my-God-am-I-cultivated distinguished attitudes! Pestilence to conceited symbolic film-language and impervious chiffres! I'll no longer have a mind for that! Léos Carax, did you ever think about, that a dialogue in a film could be natural and vivid ??? Maybe I'm too common to understand you? Or had it been your task to confirm all the clichés of a Frenchman the world can have? Guillaume the to-be-guilliotined comes to his home-palace, Mme. Deneuve, not in the picture, plays the flute: " Here am I, darling! " In this moment, I knew, that she's in the bathtub, and we `ll see her lying in there soon. Don't misunderstand me, I'm not prudish, and the incestous sex scene was the climax of the film. But this is, in Berlin, we say " etepetete ", what means something like " être-peut-être ", a snobistic, self-satisfied, and, the worst, seen that often in French movies I can tell! Other example: She, beautiful and 